# Unseen Chants: how much are we missing?
The [Cantus Index](https://cantusindex.org/) database network has been growing for almost 40 years now. Over 800,000 records is... quite a lot. And Gregorian chant is, after all, a finite tradition. Actually, it should be very finite: there are only so many positions for chants in the liturgical calendar, and you are supposed to be using the chants as defined by the Roman Rite, primarily. Places do develop their peculiarities -- local saints, or saints specific to monastic orders, or 'foundations' in the later Middle Ages that paid for a specific feast related e.g. to the profession they represented -- and new feasts were sometimes introduced even to the Roman rite, but in principle chant repertoire was limited.

From this follows that as we catalogue more and more sources, at some point we are going to run into 'diminishing returns' in terms of discovering new chant repertoire. That will also mean that the database is already quite representative of all the repertoire that the tradition contained. Of course we will never see everything (also because we only have about 30,000 extant sources from an estimated 500,000-1,500,000 medieval chant manuscripts produced), but when it takes e.g. 1,000 catalogued chants on average to find one for which we have no Cantus ID, we can be pretty sure we represent the repertoire quite well.

The problem of guessing how much repertoire we haven't seen yet -- and, conversely, how much we have already documented -- is structurally very similar to the issue that ecologists have when documenting biodiversity. The first paper on this is from 1943. This was a butterfly study in then-British Malaya. They caught a bunch of different butterfly species: from a few species there were a lot of individuals, but a lot of species were rather rare, and the biologist in charge noticed that this meant there must have been a lot of butterfly species which were actually there but were never caught. In order to get a good estimate of the actual butterfly biodiversity in the area, the number of butterfly species that were not observed -- but quite certainly are there -- must be estimated and added to the number of observed species. And this can be a lot -- many species are comparatively rare pretty much anywhere you look. That's sort of the 80-20 rule, and the phenomenon of long-tail distributions (that we saw for instance in the alleluia chants).

This class of problems is called the Unseen Species problem. It has been studied extensively in ecology, and analogies were found in other fields such as natural language processing (allocating probability for out-of-vocabulary words in language models) or literary studies (how many words did Shakespeare know?). In many real-world problems the estimated number of unseen species can be greater than the number of observed species.

We apply this for the first time to musical data -- Gregorian chant. The basic idea is very simple. Each Cantus ID is a species, and each chant record is like a butterfly caught in a net. We then apply some estimators of how many species we haven't observed yet. This will give us a principled guess of how much repertoire we can still expect to discover.

(As with other chant research, we should do this by individual chant genres.)

### General code

In [17]:
import pycantus.data as data

#### Load dataset
We are going to use [Cantus Corpus v1.0](https://github.com/DvorakovaA/CantusCorpus/tree/main/cantuscorpus_1.0) dataset, that can be loaded from available datasets of PyCantus.

In [18]:
cantuscorpus = data.load_dataset("cantuscorpus_v1.0", load_editable=True, 
                                  create_missing_sources=True)
print(f'Number of chants in corpus before any processing: {len(cantuscorpus.chants)}')
print(f'Number of sources in corpus before any processing: {len(cantuscorpus.sources)}')

Loading chants and sources...
Creating missing sources...
0 missing sources created!
Data loaded!
Number of chants in corpus before any processing: 888010
Number of sources in corpus before any processing: 2278


## Method 1: The Chao estimator

Many Unseen Species estimators exist, but for our purposes we use one by Anne Chao et al. from 1984 (and very nicely re-framed in a 2017 paper from her group). This estimator has the advantage that is is *non-parametric*: it works regardless of the underlying distribution of species (Cantus ID) frequency classes. For instance, with the communio, or antiphon and responsory frequency distributions we were plotting earlier, we don't have to care about those weird bumps in the curve.

The estimator is also very simple: all it needs are the count of singletons, and the count of doubletons: how many species only occur once, and how many occur twice in a sample.

The actual formula for the Chao1 estimator is:

\begin{align}
S = S_{obs} + \frac{f_1^2}{2 f_2}
\end{align}

where $S$ is the total expected number of species, $S_{obs}$ is the number of observed species (Cantus IDs), $f_1$ is the number of Cantus IDs observed once, and $f_2$ is the number of Cantus IDs observed twice.

It really is that simple. The reasons *why* such a simple formula works for a pretty hard problem are more complicated, of course, but the intuition is that the approximate shape of the 'long tail' of rare species continues roughly along the same curve one more step further. (Recall the logarithmic zig-zag-ish plots and imagine another step to the bottom right...)

A nice constructive proof follows from Good-Turing smoothing of language models, but we won't go into that now. What does follow from the proof, however, is that the Chao1 estimator is a **lower bound**. Under some conditions (when the shape of the 'long tail' observations is representative, which is when the sampling process is done in an unbiased way), it is also an unbiased estimate, but these conditions practically never really happen.

So, we have to be careful in interpreting the result of the estimator: if the number that falls out of the formula is e.g. 1225, it does not mean we probably haven't seen 1225 Cantus IDs, give or take. It means that we haven't seen *at least* 1225 Cantus IDs, give or take (there is always some inherent uncertainty).

Once we have the estimated $S$, we can also compute the **species coverage**: what proportion of the expected total number of species in the ecosystem -- Cantus IDs -- have we already observed? This is simply done as $S_{obs} / S$. But again we must be careful with the interpretation: since $S$ is a lower bound, this coverage is an **upper bound**. We have seen *at most* $S_{obs} / S$ of the total (bio)diversity of the ecosystem we are sampling from.

**Abundance and Incidence.**

There are two basic settings in which to do unseen species estimates: abundance, and incidence.

In case of **abundance**, we count each occurrence of each species and get our $f_1$ and $f_2$ singleton and doubleton counts simply from that.

In case we use **incidence**, we instead look at presence or absence of species in a sample. In our case, that would mean we don't care how many times a chant has occured in a source, we only care whether it was there. I think this is a more appropriate model of what is going on in chant data -- it is already pre-structured into individual samples, the samples are roughly comparable (liturgy for a year), and importantly the re-use of a chant in one place multiple times does *not* mean it is more likely that we are also going to see this chant in other sources. (Often, for a feast that is e.g. unique to a source, like a rare local saint, you would see the same antiphon used multiple times within that saint's day.)

So, in the incidence-based setting, we would get our $f_1$ as the number of Cantus IDs that only occur in one source, and $f_2$ is the number of Cantus IDs that occurs in two sources. This approach is what we use in this study, and this is also why the term 'incidence' will keep popping up in variable names or function parameters.

Experiment 1
------------

Now that we have defined the method, and the material, we can run the 'unseen chants' experiment.

This experiment is from submission of Jan Hajič jr. and Fabian Moss to DLfM 2025 (Knowing when to stop: insights from ecology for building catalogues, collections, and corpora, [arXiv](https://www.arxiv.org/abs/2507.14614)).  
This class of models is based off of the work of Kestemont et al., 2022, who did this for Dutch and other chivalric epics. Mike Kestemont and his co-authors implemented this in a convenient Python library called `copia` (from: 'cornucopia'), which we are going to re-use.

In [ ]:
!pip install git+https://github.com/mikekestemont/copia.git

In [39]:
# Importing the essential functions from Copia
from copia.data import to_copia_dataset

# Importing useful libraries
import numpy as np  # This is an important library for all sorts of fast math.
import os           # This is a library for traversing and manipulating the file system.
import copy        # This library helps with copying data structures.
import collections  # This library has some useful data structures.
import pprint     # This library helps with pretty-printing data structures.

# Importing useful parts of PyCantus
from pycantus.filtration import  Filter

We have already loaded all our chant data, we just have to look at how to format them to be able to use the implementation of unseen species estimators that are available in the `copia` library.

We will cheat here a little and just tell you that all we need is a list of pairs: `(species, sample)`. In our case, the `species` is the `cantus_id` field of a chant, and the `sample` corresponds to the `source_id` in which we have observed that particular chant.

We would like to limit ourselves to the core genres for the Office and Mass.

In [21]:
GENRES_OFFICE = ['A', 'R', 'V', 'W', 'I']
GENRES_MASS_PROPERS = ['In', 'InV', 'Gr', 'GrV', 'Al', 'AlV', 'Of', 'OfV', 'Cm', 'CmV', 'Tc', 'TcV']


In [28]:
observation_lists_per_genre = {}
for genre in GENRES_OFFICE + GENRES_MASS_PROPERS:
    # Prepare filter by genre
    genre_filter = Filter('filter_'+genre)
    genre_filter.add_value_include('genre', genre)
    # Apply filter to a copy of the corpus
    filtered_corpus = copy.copy(cantuscorpus)
    filtered_corpus.apply_filter(genre_filter)
    # Now we can make the observation list for this genre
    observation_lists_per_genre[genre] = []
    for ch in filtered_corpus.chants:
        observation_lists_per_genre[genre].append( (ch.cantus_id, ch.srclink) )

# Reporting again
_total_observations = sum([len(observations_list) for observations_list in observation_lists_per_genre.values()])
print('Made observation lists for {} genres; {} observations total.'.format(len(observation_lists_per_genre), _total_observations))

Made observation lists for 17 genres; 796456 observations total.


What are the basic descriptive statistics for each genre?

In [30]:
def extract_incidence_statistics(observation_list):
    all_species = collections.defaultdict(int)
    all_samples = collections.defaultdict(int)
    for (sp, sam) in observation_list:
        all_species[sp] += 1
        all_samples[sam] += 1

    n_singletons = len([sp for sp in all_species if all_species[sp] == 1])
    #n_doubletons = len([sp for sp in all_species if all_species[sp] == 2])

    stats = {
        'n_species': len(all_species),
        'n_samples': len(all_samples),
    }
    return stats

In [31]:
stats_per_genre = {g: extract_incidence_statistics(observation_lists_per_genre[g])
                   for g in observation_lists_per_genre}

Now that we have observation lists, we can feed them into `copia` and run the unseen species estimators!

We first wrap the observation lists with some miscellaneous data structure defined by `copia`. The important part of this is defining the `data_type` as `'incidence'`, to reflect the decision to use incidence rather than abundance.

In [23]:
copia_datasets_per_genre = {g: to_copia_dataset(observation_lists_per_genre[g],
                                                data_type="incidence",
                                                input_type='observation_list',
                                                n_sampling_units=1)
                            for g in observation_lists_per_genre}

Copia supports more estimators than just Chao1, and in a bigger study we would want to compare them, so we prepare the function to run different unseen species estimates on the same data:

In [24]:
from copia.estimators import diversity
# This is the interface copia provides for actually computing the unseen species estimates.


def extract_incidence_diversities(incidences,         # The input data, as a copia dataset
                                  factor_max_steps=8, # Nevermind this
                                  n_iter=20,          # and this
                                  step_size=10,       # and this,
                                  methods=('chao1', 'ichao1', 'ace', 'jackknife', 'egghe_proot'),   # These are some supported estimators,
                                  **kwargs):          # and this.
    # We store the results in a dictionary (again, this is the 'shortcut' dictionary construction syntax)
    diversities = {method: round(diversity(incidences,
                                           method=method))
                   for method in methods}
    return diversities


and now we uncermoniously compute just the Chao1 values:

In [25]:
diversities_per_genre = {g: extract_incidence_diversities(copia_datasets_per_genre[g], methods=['chao1'])
                         for g in copia_datasets_per_genre}

### Results 1

Look what we found!

In [26]:
diversities_per_genre

{'A': {'chao1': 19032},
 'R': {'chao1': 8968},
 'V': {'chao1': 16018},
 'W': {'chao1': 1444},
 'I': {'chao1': 981},
 'In': {'chao1': 525},
 'InV': {'chao1': 1300},
 'Gr': {'chao1': 343},
 'GrV': {'chao1': 408},
 'Al': {'chao1': 1407},
 'AlV': {'chao1': 398},
 'Of': {'chao1': 387},
 'OfV': {'chao1': 580},
 'Cm': {'chao1': 384},
 'CmV': {'chao1': 951},
 'Tc': {'chao1': 141},
 'TcV': {'chao1': 417}}

These numbers are the lower bound on $f_0$, the number of unseen Cantus IDs for each genre.

Now in order to properly understand these numbers, we need to report them in relation to other statistics about each genre. Specifically, we want the coverage, and we want to see how the proportion of unseen Cantus IDs -- basically, the diversity of that genre -- relates to the ratio between the number of distinct Cantus IDs and nubmer of sources containing that genre, which will help illustrate that this measure of diversity is not just a trivial result of this ratio, or "average density" of the genre.

In [27]:
def report_for_genre(g, stats, diversities):
    g_stats = stats[g]
    n_species, n_samples = g_stats['n_species'], g_stats['n_samples']

    g_diversities = diversities[g]
    g_diversity_ratios = {k: n_species / v for k, v in g_diversities.items()}
    diversities_string = ', '.join(['{}: {}\tcov. (max.): {:.3f}'.format(k, v, g_diversity_ratios[k]) for k, v in g_diversities.items()])

    print('{}:\t{} species\t{} samples\t{}'.format(g, n_species, n_samples, diversities_string))


In [32]:
for g in diversities_per_genre:
  report_for_genre(g, stats_per_genre, diversities_per_genre)

A:	12667 species	1347 samples	chao1: 19032	cov. (max.): 0.666
R:	5790 species	1088 samples	chao1: 8968	cov. (max.): 0.646
V:	9382 species	1029 samples	chao1: 16018	cov. (max.): 0.586
W:	1072 species	605 samples	chao1: 1444	cov. (max.): 0.742
I:	658 species	455 samples	chao1: 981	cov. (max.): 0.671
In:	350 species	527 samples	chao1: 525	cov. (max.): 0.667
InV:	796 species	427 samples	chao1: 1300	cov. (max.): 0.612
Gr:	229 species	522 samples	chao1: 343	cov. (max.): 0.668
GrV:	292 species	476 samples	chao1: 408	cov. (max.): 0.716
Al:	946 species	506 samples	chao1: 1407	cov. (max.): 0.672
AlV:	101 species	190 samples	chao1: 398	cov. (max.): 0.254
Of:	276 species	501 samples	chao1: 387	cov. (max.): 0.713
OfV:	416 species	236 samples	chao1: 580	cov. (max.): 0.717
Cm:	320 species	501 samples	chao1: 384	cov. (max.): 0.833
CmV:	664 species	89 samples	chao1: 951	cov. (max.): 0.698
Tc:	111 species	285 samples	chao1: 141	cov. (max.): 0.787
TcV:	343 species	262 samples	chao1: 417	cov. (max.): 0.82

It would be better to report these results separately by type of liturgy.

In [33]:
print('Office genre diversities')
for g in GENRES_OFFICE:
  report_for_genre(g, stats_per_genre, diversities_per_genre)

print('\n')   # Just an empty line
print('Mass genre diversities')
for g in GENRES_MASS_PROPERS:
  report_for_genre(g, stats_per_genre, diversities_per_genre)


Office genre diversities
A:	12667 species	1347 samples	chao1: 19032	cov. (max.): 0.666
R:	5790 species	1088 samples	chao1: 8968	cov. (max.): 0.646
V:	9382 species	1029 samples	chao1: 16018	cov. (max.): 0.586
W:	1072 species	605 samples	chao1: 1444	cov. (max.): 0.742
I:	658 species	455 samples	chao1: 981	cov. (max.): 0.671


Mass genre diversities
In:	350 species	527 samples	chao1: 525	cov. (max.): 0.667
InV:	796 species	427 samples	chao1: 1300	cov. (max.): 0.612
Gr:	229 species	522 samples	chao1: 343	cov. (max.): 0.668
GrV:	292 species	476 samples	chao1: 408	cov. (max.): 0.716
Al:	946 species	506 samples	chao1: 1407	cov. (max.): 0.672
AlV:	101 species	190 samples	chao1: 398	cov. (max.): 0.254
Of:	276 species	501 samples	chao1: 387	cov. (max.): 0.713
OfV:	416 species	236 samples	chao1: 580	cov. (max.): 0.717
Cm:	320 species	501 samples	chao1: 384	cov. (max.): 0.833
CmV:	664 species	89 samples	chao1: 951	cov. (max.): 0.698
Tc:	111 species	285 samples	chao1: 141	cov. (max.): 0.787
TcV:	34

And we should actually also include results for the aggregated chants of all genres for a given liturgy:

In [34]:
mass_observations_list = []
for g in GENRES_MASS_PROPERS:
    mass_observations_list.extend(observation_lists_per_genre[g])
office_observations_list = []
for g in GENRES_OFFICE:
    office_observations_list.extend(observation_lists_per_genre[g])

Because it would be a bit annoying to type out all the individual steps for running the experiment, we can re-write the whole experiment in a function:

In [36]:
def compute_result_from_observations_list(observations_list,
                                          n_bootstrap=None):
    """If n_boostrap > 0, the results will be lists for each key (n_species, n_samples, chao1, coverage)"""
    # Copia has some bug in incidence bootstrapping.
    # CI = False
    # if n_iter and (n_iter > 1):
    #     CI = True
    result = {}

    # No bootstrapping
    stats = extract_incidence_statistics(observations_list)
    copia_dataset = to_copia_dataset(observations_list,
                                      data_type="incidence",
                                      input_type='observation_list',
                                      n_sampling_units=1)
    diversities = extract_incidence_diversities(copia_dataset,
                                                methods=['chao1'],
                                                CI=False)

    coverage = stats['n_species'] / diversities['chao1']
    result = {}
    for k, v in stats.items(): result[k] = v
    for k, v in diversities.items(): result[k] = v
    result['coverage'] = coverage

    return result

...and then running the whole experiment from an observations list becomes a single line of code:

In [40]:
mass_results = compute_result_from_observations_list(mass_observations_list)
print('Mass results: {}'.format(pprint.pformat(mass_results)))
print('\n')
office_results = compute_result_from_observations_list(office_observations_list)
print('Office results: {}'.format(pprint.pformat(office_results)))

Mass results: {'chao1': 6471,
 'coverage': 0.7241539174779786,
 'n_samples': 779,
 'n_species': 4686}


Office results: {'chao1': 45676,
 'coverage': 0.6436202819861634,
 'n_samples': 1549,
 'n_species': 29398}


### Discussion 1

What we can immediately notice is that while there is orders of magnitude fewer records for mass propers, the upper bound on coverage is higher (0.72) than for the office (0.64). This tracks with the observation from chant scholarship that mass repertoire is much more fixed -- less diverse -- than repertoire for the office. In terms of documenting chant diversity, the focus of the Cantus Database on cataloguing office sources is well placed: there is apparently more to find there.

In [41]:
# We re-print the per-genre report, just for convenience
print('Office genre diversities')
for g in GENRES_OFFICE:
  report_for_genre(g, stats_per_genre, diversities_per_genre)

print('\n')   # Just an empty line
print('Mass genre diversities')
for g in GENRES_MASS_PROPERS:
  report_for_genre(g, stats_per_genre, diversities_per_genre)


Office genre diversities
A:	12667 species	1347 samples	chao1: 19032	cov. (max.): 0.666
R:	5790 species	1088 samples	chao1: 8968	cov. (max.): 0.646
V:	9382 species	1029 samples	chao1: 16018	cov. (max.): 0.586
W:	1072 species	605 samples	chao1: 1444	cov. (max.): 0.742
I:	658 species	455 samples	chao1: 981	cov. (max.): 0.671


Mass genre diversities
In:	350 species	527 samples	chao1: 525	cov. (max.): 0.667
InV:	796 species	427 samples	chao1: 1300	cov. (max.): 0.612
Gr:	229 species	522 samples	chao1: 343	cov. (max.): 0.668
GrV:	292 species	476 samples	chao1: 408	cov. (max.): 0.716
Al:	946 species	506 samples	chao1: 1407	cov. (max.): 0.672
AlV:	101 species	190 samples	chao1: 398	cov. (max.): 0.254
Of:	276 species	501 samples	chao1: 387	cov. (max.): 0.713
OfV:	416 species	236 samples	chao1: 580	cov. (max.): 0.717
Cm:	320 species	501 samples	chao1: 384	cov. (max.): 0.833
CmV:	664 species	89 samples	chao1: 951	cov. (max.): 0.698
Tc:	111 species	285 samples	chao1: 141	cov. (max.): 0.787
TcV:	34

While the office genres show a similar level of diversity, for genres of mass propers the upper bound on coverage fluctuates *wildly*. Even though alleluias are typically considered to be the most diverse of the main mass proper genres (i.e. without taking verses into account), we get a coverage up to 0.672. 

## Method 2: Accumulation curve

The **accumulation curve** tracks the relationship between how many specimen we have observed, and how many species in total we have seen. This is typically a concave function -- at the beginning, almost every specimen is from a species we haven't seen yet, so the accumulation curve grows very quickly. But as we keep sampling, probably we run into the pesky frequent species over and over, and even the somewhat less frequent ones we will have already seen many times, so the interval between finding new species is going to increase. This is the point of 'diminishing returns' of the cataloguing effort, in terms of diversity.

So, have we actually reached the point of diminishing returns? How many new "species" can we expect per e.g. 100 catalogued chants? (Or: what is the probability that the next chant we catalogue will require a new Cantus ID?)

Fortunately, the Cantus database does record for many of its sources the indexing year, so we can actually reconstruct the empirical accumulation curve, and we can track the probability of encountering a new species when cataloguing over time. 